<a href="https://colab.research.google.com/github/CodedSpirit-dev/ModeloCovidMX19/blob/main/PTfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
! git clone https://github.com/rapidsai/rapidsai-csp-utils.git
! python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.


Traceback (most recent call last):
  File "/home/codedspirit/ModeloCovidMX19/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [3]:
#! wget https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

--2023-11-09 00:28:22--  https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 201.98.60.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|201.98.60.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18045440 (17M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip.2’

datos_abiertos_covi 100%[===================>]  17.21M   459KB/s    in 8.1s    

2023-11-09 00:28:30 (2.13 MB/s) - ‘datos_abiertos_covid19.zip.2’ saved [18045440/18045440]



In [4]:
#! unzip datos_abiertos_covid19.zip

/bin/bash: line 1: unzip: command not found


In [5]:
import numpy as np
import pandas as pd
import cudf


data=cudf.read_csv('COVID19MEXICO.csv')
data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 40 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   FECHA_ACTUALIZACION    1186694 non-null  object
 1   ID_REGISTRO            1186694 non-null  object
 2   ORIGEN                 1186694 non-null  int64
 3   SECTOR                 1186694 non-null  int64
 4   ENTIDAD_UM             1186694 non-null  object
 5   SEXO                   1186694 non-null  int64
 6   ENTIDAD_NAC            1186694 non-null  object
 7   ENTIDAD_RES            1186694 non-null  object
 8   MUNICIPIO_RES          1186694 non-null  object
 9   TIPO_PACIENTE          1186694 non-null  int64
 10  FECHA_INGRESO          1186694 non-null  object
 11  FECHA_SINTOMAS         1186694 non-null  object
 12  FECHA_DEF              1186694 non-null  object
 13  INTUBADO               1186694 non-null  int64
 14  NEUMONIA               1186694 non-null

In [6]:
import cudf
import cuml

# Cargamos los datos en un DataFrame de cuDF
data = cudf.read_csv('COVID19MEXICO.csv')

# Eliminamos las columnas que no necesitamos
df = data.drop(columns=['SECTOR', 'MUNICIPIO_RES', 'HABLA_LENGUA_INDIG', 'ID_REGISTRO', 'ORIGEN', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'INDIGENA', 'ENTIDAD_UM', 'ENTIDAD_NAC', 'NACIONALIDAD', 'ENTIDAD_RES'])
print(df.columns)

# Convertimos las columnas de fecha
df['FECHA_ACTUALIZACION'] = cudf.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_INGRESO'] = cudf.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS'] = cudf.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_DEF'] = cudf.to_datetime(df['FECHA_DEF'], errors='coerce')

# Convertimos tipos de datos
df['TIPO_PACIENTE'] = df['TIPO_PACIENTE'].astype('str')

# Codificación de etiquetas
for col in ['TIPO_PACIENTE', 'SEXO']:
  df[col], _ = df[col].factorize()

# Creación de características a partir de fechas
df['DIAS_DEFUNCION'] = (df['FECHA_ACTUALIZACION'] - df['FECHA_DEF']).dt.days
df['DIAS_HOSPITALIZACION'] = (df['FECHA_ACTUALIZACION'] - df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS'] = (df['FECHA_ACTUALIZACION'] - df['FECHA_SINTOMAS']).dt.days

# Limpieza de datos
df['DIAS_DEFUNCION'] = df['DIAS_DEFUNCION'].apply(lambda x: 0 if x < 0 else x)

# Eliminamos más columnas que no vamos a necesitar
df = df.drop(columns=['FECHA_ACTUALIZACION', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'DIAS_DEFUNCION', 'MIGRANTE'])

# Preparamos los datos para el modelo
X = df.drop(columns=['CLASIFICACION_FINAL'])
y = df['CLASIFICACION_FINAL']

# Ahora X e y están listos para ser utilizados en operaciones que utilicen la GPU


Index(['FECHA_ACTUALIZACION', 'SEXO', 'TIPO_PACIENTE', 'FECHA_INGRESO',
       'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA', 'EDAD',
       'EMBARAZO', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION',
       'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'UCI'],
      dtype='object')


In [7]:
from cuml.model_selection import train_test_split

# Suponiendo que X e y son cuDF DataFrame/Series y se han definido anteriormente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 25 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   SEXO                   1186694 non-null  int8
 1   TIPO_PACIENTE          1186694 non-null  int8
 2   INTUBADO               1186694 non-null  int64
 3   NEUMONIA               1186694 non-null  int64
 4   EDAD                   1186694 non-null  int64
 5   EMBARAZO               1186694 non-null  int64
 6   DIABETES               1186694 non-null  int64
 7   EPOC                   1186694 non-null  int64
 8   ASMA                   1186694 non-null  int64
 9   INMUSUPR               1186694 non-null  int64
 10  HIPERTENSION           1186694 non-null  int64
 11  OTRA_COM               1186694 non-null  int64
 12  CARDIOVASCULAR         1186694 non-null  int64
 13  OBESIDAD               1186694 non-null  int64
 14  RENAL_CRONICA          1186694 non-null  int64
 15

# Explorar datos

In [ ]:
# Hacer scatter     
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Crear matriz de correlación con Pandas
corr_matrix = df.corr().abs()

# Imprimir la corr
print(corr_matrix)

: 

In [ ]:
import cudf
import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import accuracy_score

clf = RandomForestClassifier(random_state=50)
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


0.9903639554977417


In [ ]:
import cuml
from cuml.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=50)
#
X_train=X_train.astype('float32')
y_train=y_train.astype('float32')

clf.fit(X_train.to_numpy(),y_train.to_numpy())
y_pred=clf.predict(X_test.to_numpy())

print(classification_report(y_test.to_numpy(),y_pred))


              precision    recall  f1-score   support

         1.0       1.00      0.01      0.01      1560
         2.0       0.00      0.00      0.00         7
         3.0       1.00      1.00      1.00     83886
         4.0       0.00      0.00      0.00        36
         5.0       0.61      0.25      0.36       410
         6.0       0.95      0.97      0.96     12835
         7.0       0.99      1.00      0.99    138604

    accuracy                           0.99    237338
   macro avg       0.65      0.46      0.47    237338
weighted avg       0.99      0.99      0.99    237338



/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

In [ ]:
import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier


cuml_RFC=RandomForestClassifier(random_state=42)
cuml_KNC=KNeighborsClassifier()

# Fit the random forest classifier on the training data
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
cuml_RFC.fit(X_train, y_train)

# Fit the KNN classifier on the training data
cuml_KNC.fit(X_train, y_train)




classifiers=[
    {
        'Ramdom Forest': cuml_RFC
    },
    {
        'KNN': cuml_KNC

    }
]

for classifier in classifiers:
    for name, clf in classifier.items():
        y_pred = clf.predict(X_test).to_pandas().values
        print(classification_report(y_test.to_pandas().values, y_pred))
        print(classification_report(y_test.to_numpy(), y_pred))




/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

              precision    recall  f1-score   support

         1.0       1.00      0.01      0.01      1560
         2.0       0.00      0.00      0.00         7
         3.0       1.00      1.00      1.00     83886
         4.0       0.00      0.00      0.00        36
         5.0       0.67      0.24      0.35       410
         6.0       0.95      0.97      0.96     12835
         7.0       0.99      1.00      0.99    138604

    accuracy                           0.99    237338
   macro avg       0.66      0.46      0.47    237338
weighted avg       0.99      0.99      0.99    237338



/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

              precision    recall  f1-score   support

         1.0       1.00      0.01      0.01      1560
         2.0       0.00      0.00      0.00         7
         3.0       1.00      1.00      1.00     83886
         4.0       0.00      0.00      0.00        36
         5.0       0.67      0.24      0.35       410
         6.0       0.95      0.97      0.96     12835
         7.0       0.99      1.00      0.99    138604

    accuracy                           0.99    237338
   macro avg       0.66      0.46      0.47    237338
weighted avg       0.99      0.99      0.99    237338



/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

              precision    recall  f1-score   support

         1.0       0.18      0.01      0.02      1560
         2.0       0.00      0.00      0.00         7
         3.0       0.95      0.85      0.90     83886
         4.0       0.00      0.00      0.00        36
         5.0       0.45      0.03      0.06       410
         6.0       0.96      0.92      0.94     12835
         7.0       0.90      0.97      0.93    138604

    accuracy                           0.92    237338
   macro avg       0.49      0.40      0.41    237338
weighted avg       0.91      0.92      0.91    237338

              precision    recall  f1-score   support

         1.0       0.18      0.01      0.02      1560
         2.0       0.00      0.00      0.00         7
         3.0       0.95      0.85      0.90     83886
         4.0       0.00      0.00      0.00        36
         5.0       0.45      0.03      0.06       410
         6.0       0.96      0.92      0.94     12835
         7.0       0.90 

/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
! pip install gradio

In [ ]:
def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo, diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
            migrante, uci, dias_hospitalizacion, dias_sintomas) :
    x = np.array([[sexo, tipo_paciente, intubado, neumonia, edad, embarazo, diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
            migrante, uci, dias_hospitalizacion, dias_sintomas]])
    prediction = clf.predict(x)
    return prediction


In [ ]:
import gradio as gr
import pandas as pd
import numpy as np


def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
             diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
           uci, dias_hospitalizacion, dias_sintomas):


    # Crea un DataFrame con los valores de entrada
    input_data = pd.DataFrame([[
        sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
       diabetes, epoc, asma, inmusupr,
        hipertension, otra_com, cardiovascular, obesidad,
        renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
        resultado_lab, toma_muestra_antigeno, resultado_antigeno, uci, dias_hospitalizacion, dias_sintomas
        ]], columns=[
        'SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO','DIABETES', 'EPOC',
        'ASMA', 'INMUSUPR','HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD','RENAL_CRONICA',
        'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB','RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO',
        'RESULTADO_ANTIGENO','UCI', 'DIAS_HOSPITALIZACION', 'DIAS_SINTOMAS'])

    prediction = clf.predict(input_data)
    return prediction




gr.Row
inputs = [
    gr.Radio(choices=[0, 1], label='Sexo'),
    gr.Radio(choices=[0, 1], label='Tipo de Paciente'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Intubado'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Neumonía'),
    gr.Slider(minimum=0, maximum=120, value=30, label='Edad'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Embarazo'),
    gr.Radio(choices=[1, 2, 98], label='Diabetes'),
    gr.Radio(choices=[1, 2, 98], label='EPOC'),
    gr.Radio(choices=[1, 2, 98], label='Asma'),
    gr.Radio(choices=[1, 2, 98], label='Inmunosupresión'),
    gr.Radio(choices=[1, 2, 98], label='Hipertensión'),
    gr.Radio(choices=[1, 2, 98], label='Otra Comorbilidad'),
    gr.Radio(choices=[1, 2, 98], label='Cardiovascular'),
    gr.Radio(choices=[1, 2, 98], label='Obesidad'),
    gr.Radio(choices=[1, 2, 98], label='Enfermedad Renal Crónica'),
    gr.Radio(choices=[1, 2, 98], label='Tabaquismo'),
    gr.Radio(choices=[1, 2, 99], label='Contacto con otro caso'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de laboratorio'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de laboratorio'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='UCI'),
    gr.Number(label='Días de Hospitalización',value=0),
    gr.Number(label='Días con Síntomas',value=0),
]

# Crear componente de salida para Gradio
outputs = gr.Textbox(label="Predicción")

# Crear la interfaz de Gradio
demo = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)

# Ejecutar la aplicación web
if __name__ == "__main__":
    demo.launch(show_api=False,debug=True)

/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
